In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sb
plt.rcParams['figure.figsize'] = (4, 4)

# ETL (extract, transform, load)

## load csv, shp file

In [ ]:
#load into main df_csv
df_csv = pd.read_csv('rm_crop_yields_1938_2021.csv')

In [ ]:
df_csv.info()

In [ ]:
df_csv.rename(columns={"Winter Wheat": "WinterWheat", "Spring Wheat": "SpringWheat","Fall Rye":"FallRye",\
             "Canary Seed":"CanarySeed","Spring Rye":"SpringRye","Tame Hay":"TameHay"},inplace=True)

In [ ]:
#https://saskpulse.com
#https://www.rayglen.com/grain-conversion-calculator/
# (lbs/ac) -> Mustard (50 lbs/bu), Sunflower (30 lbs/bu), Lentils (60 lbs/bu), 
#             Canary Seeed (50 lbs/bu), Chickpeas (60lb/bu)

# Tame Hay (tons/ac)
# All the rest are bushel/acre (bu/ac)
df_csv['Mustard']=df_csv['Mustard']/50
df_csv['Sunflowers']=df_csv['Sunflowers']/30
df_csv['Lentils']=df_csv['Lentils']/60
df_csv['CanarySeed']=df_csv['CanarySeed']/50
df_csv['Chickpeas']=df_csv['Chickpeas']/60

In [ ]:
#load shp data
gdf = gpd.read_file('Rural Municipality.shp')

#drop columns that won't be using
gdf.drop(['PPID','EFFDT','EXPDT','FEATURECD','SHAPE_AREA','SHAPE_LEN'],axis=1,inplace=True)

#rename column to match with main df
gdf.rename(columns=
{   'RMNO': 'RM',
    'RMNM': 'Municipality'
}, inplace=True)

#match data type between df_csv and gdf
gdf['RM']=gdf['RM'].astype('int')
gdf['Municipality']=gdf['Municipality'].astype('string')


In [ ]:
gdf.info()

# EDA (Extrapolatory Data Analysis)

#### check for unique values

In [ ]:
gdf['RM'].unique()
gdf['RM'].nunique()
#298 unique RM

#### check for duplicated values

In [ ]:
gdf.duplicated().sum()

In [ ]:
df_describe = df_csv.describe().copy()
df_describe
# Total of 25017 rows
# Year from 1938 to 2021 ~ 84 years
# 299 RM from 1 to 622 
# Spring Wheat, Barley and Oats have the most rows -> more complete data?
# Oats, Winter Wheat, Barley has the most mean -> most yield
# Tame Hay, Spring Rye, Flax have the least mean -> least yield

In [ ]:
#count unique RM
df_csv['RM'].unique()
df_csv['RM'].nunique()

In [ ]:
#check for duplicated rows
df_csv.duplicated().sum()

In [ ]:
#check how many records there are each year
#-->not all have data for all years
temp_df= df_csv.groupby('Year').count()['RM']
temp_df = temp_df[temp_df!=299]
temp_df

In [ ]:
#check for RM with less than 84 years of data
temp_df= df_csv.groupby('RM').count()['Year']
temp_df = temp_df[temp_df!=84]
temp_df

In [ ]:
#Municipality with less than 84 years of data

pd.merge(temp_df,gdf,on='RM')

In [ ]:
#check for RM not in geodata

temp_df= df_csv.groupby('RM').count().index
temp_df = temp_df[~temp_df.isin(gdf['RM'])]
temp_df

#278 Kutawa, Prairie No. 408, Greenfield No. 529

In [ ]:
#check for geodata RM not in main data set

temp_df= df_csv.groupby('RM').count().index
temp_gdf = gdf[~gdf['RM'].isin(temp_df)]
temp_gdf

## Time Series Analysis

In [ ]:
df_csv.groupby('RM').mean().sort_values('SpringWheat',ascending=False)

In [ ]:
df_csv.groupby('RM').mean().sort_values('Barley',ascending=False)

In [ ]:
df_csv.groupby('RM').mean().sort_values('Oats',ascending=False)

## Forecast for RM429

In [ ]:
df_RM_429 = df_csv.loc[df_csv['RM']==429][['Year','SpringWheat','Barley','Oats']]

In [ ]:
df_RM_SpringWheat =df_csv.loc[df_csv['RM']==429][['Year', 'SpringWheat']]
df_RM_Barley =df_csv.loc[df_csv['RM']==429][['Year', 'Barley']]
df_RM_Oats =df_csv.loc[df_csv['RM']==429][['Year', 'Oats']]

In [ ]:
df_RM_SpringWheat['Year'] = pd.to_datetime(df_RM_SpringWheat['Year'], format='%Y')
df_RM_SpringWheat = df_RM_SpringWheat.set_index('Year')

df_RM_Barley['Year'] = pd.to_datetime(df_RM_Barley['Year'], format='%Y')
df_RM_Barley = df_RM_Barley.set_index('Year')

df_RM_Oats['Year'] = pd.to_datetime(df_RM_Barley['Year'], format='%Y')
df_RM_Oats = df_RM_Barley.set_index('Year')

In [ ]:
df_RM_429['Year'] = pd.to_datetime(df_RM_429['Year'], format='%Y')
df_RM_429 = df_RM_429.set_index('Year')
df_RM_429

In [ ]:
#fig, ax = plt.subplots(figsize=(6,6))

plt.plot(df_RM_429.index,df_RM_429['SpringWheat'], lw=0.8, color="blue", label="Spring Wheat")
plt.plot(df_RM_429.index,df_RM_429['Barley'], lw=0.8, color="orange", label="Barley")
plt.plot(df_RM_429.index,df_RM_429['Oats'], lw=0.8, color="green", label="Oats")


plt.legend()
plt.show

In [ ]:
X = df_RM_429['SpringWheat']
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))

# Time Forecasting Model

#### Mean Absolute Percentage Error

In [ ]:
def mape(y_true, y_pred):
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
    

#### Ad Fuller Test

In [ ]:
# Import adfuller
from statsmodels.tsa.stattools import adfuller

def adf_test(df_test):
    result = adfuller(df_RM_429['SpringWheat'])
    print('ADF Test Statistic: %.2f' % result[0])
    print('#Lags Used: '+str(result[2]))
    print('5%% Critical Value: %.2f' % result[4]['5%'])
    print('p-value: %.2f' % result[1])


In [ ]:
adf_test(df_RM_429['SpringWheat'])

#### Seasonal Decompose

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df_RM_429['SpringWheat'])  
figure = plt.figure()  
figure = decomposition.plot()  
figure.set_size_inches(15, 8)

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=2).mean()
    rolstd = timeseries.rolling(window=2).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
# test_stationarity(df_RM_429['SpringWheat'])

#### Train and Test data split at 2011

In [ ]:
train = df_RM_429[:'2010-01-01']['SpringWheat']    # until 2010
len(train)

test = df_RM_429['2011-01-01':]['SpringWheat']     # from 2011
len(test) 

## Simple Exponential Smoothing

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing
#import plotly.express as px

alpha = 0.8
ses = SimpleExpSmoothing(df_RM_429['SpringWheat'])
model = ses.fit(smoothing_level = alpha, optimized = False)

forecast = model.forecast(5)

forecast

In [ ]:
fit1 = SimpleExpSmoothing(df_RM_429['SpringWheat'], initialization_method="heuristic").fit(
    smoothing_level=0.2, optimized=False
)
fcast1 = fit1.forecast(3).rename(r"$\alpha=0.2$")
fit2 = SimpleExpSmoothing(df_RM_429['SpringWheat'], initialization_method="heuristic").fit(
    smoothing_level=0.6, optimized=False
)
fcast2 = fit2.forecast(3).rename(r"$\alpha=0.6$")
fit3 = SimpleExpSmoothing(df_RM_429['SpringWheat'], initialization_method="estimated").fit()
fcast3 = fit3.forecast(3).rename(r"$\alpha=%s$" % fit3.model.params["smoothing_level"])

plt.figure(figsize=(12, 8))
plt.plot(df_RM_429['SpringWheat'], marker="o", color="black")
plt.plot(fit1.fittedvalues, marker="o", color="blue")
(line1,) = plt.plot(fcast1, marker="o", color="blue")
plt.plot(fit2.fittedvalues, marker="o", color="red")
(line2,) = plt.plot(fcast2, marker="o", color="red")
plt.plot(fit3.fittedvalues, marker="o", color="green")
(line3,) = plt.plot(fcast3, marker="o", color="green")
plt.legend([line1, line2, line3], [fcast1.name, fcast2.name, fcast3.name])

In [ ]:
from sklearn.metrics import mean_absolute_error

ses_model = SimpleExpSmoothing(train).fit(smoothing_level=0.2)

y_pred = ses_model.forecast(len(test))

ses_mape = mape(test, y_pred)

print(f'Simple Exponential Smoothing: {ses_mape}%')


In [ ]:
train.plot(title="Spring Wheat (Single Exponential Smoothing)",label='train')
test.plot(label='test')
y_pred.plot(label='pred')
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

## Double Exponential Smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
double_model = ExponentialSmoothing(train,trend='add').fit()
y_pred = double_model.forecast(11).rename('DES Forecast')

In [ ]:
#train_data['#Passengers'].plot(legend=True,label='TRAIN')
#test_data['#Passengers'].plot(legend=True,label='TEST',figsize=(12,8))
#test_predictions.plot(legend=True,label='PREDICTION');

plt.plot(train,color='blue',label='Train',title = 'Double Exponential Smoothing' )
plt.plot(test, color='red',label='Test')
plt.plot(y_pred,color='green',label='Forecast')
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
ar_mape = mape(test, y_pred)
print(f'Double Exponential Smoothing: {ar_mape}%')

## Triple Exponential Smoothing

In [ ]:
triple_model = ExponentialSmoothing(train,trend='add',seasonal='add',seasonal_periods=12).fit()
y_pred = triple_model.forecast(11).rename('TES Forecast')

In [ ]:
#train_data['#Passengers'].plot(legend=True,label='TRAIN')
#test_data['#Passengers'].plot(legend=True,label='TEST',figsize=(12,8))
#test_predictions.plot(legend=True,label='PREDICTION');

plt.plot(train,color='blue',label='Train',title='Triple Exponential Smoothing' )
plt.plot(test, color='red',label='Test')
plt.plot(y_pred,color='green',label='Forecast')
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
ar_mape = mape(test, y_pred)
print(f'Triple Exponential Smoothing: {ar_mape}%')

## Autoregression

In [ ]:
from pandas.plotting import lag_plot
lag_plot(df_RM_429['SpringWheat'])
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df_RM_429['SpringWheat'])

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

# train autoregression
ar_model = AutoReg(train.dropna(),lags=1).fit()
print(ar_model.summary())

In [ ]:
#
# Make the predictions
#
y_pred = ar_model.predict(start=len(train), end=(len(df_RM_429)-1), dynamic=False)
#
# Plot the prediction vs test data
#
from matplotlib import pyplot
plt.plot(train,color='blue',label='Train' )
plt.plot(test, color='red',label='Test')
plt.plot(y_pred,color='green',label='Forecast')
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
ar_mape = mape(test, y_pred)
print(f'Autogression: {ar_mape}%')

## Moving Average

In [ ]:
y_pred = df_RM_429['SpringWheat'].copy()
ma_window = 2
y_pred = df_RM_429['SpringWheat'].rolling(ma_window).mean()
y_pred[len(train):] = y_pred[len(train)-1]
y_pred

In [ ]:
plt.figure(figsize=(20,5))
plt.grid()
plt.plot(train, label='Train',color='blue')
plt.plot(test, label='Test',color='red')
plt.plot(y_pred, label='Forecast',color='green')
plt.legend(loc='best')
plt.title('Simple Moving Average Method')
plt.show()

In [ ]:
ar_mape = mape(test, y_pred)
print(f'Moving Average: {ar_mape}%')